In [1]:
from kafka import KafkaConsumer
from json import loads
import logging
import joblib
import pandas as pd
from sklearn.linear_model import LinearRegression
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import json
import mysql.connector

In [82]:
model = joblib.load('../Models/regresion_lineal.pkl')

In [83]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

In [84]:
import json
from json import loads

consumer = KafkaConsumer(
    'w',
    bootstrap_servers='localhost:9092',
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
     consumer_timeout_ms=10000,
    auto_offset_reset='earliest',
    enable_auto_commit=True)

collected_data = []

for message in consumer:
    data = message.value
    collected_data.append(data)
    offset = message.offset
    logging.info(f'Message received: {data}')

2024-05-24 10:41:39,032 - <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
2024-05-24 10:41:39,036 - Probing node bootstrap-0 broker version
2024-05-24 10:41:39,039 - <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: Connection complete.
2024-05-24 10:41:39,149 - Broker version identified as 2.6.0
2024-05-24 10:41:39,151 - Set configuration api_version=(2, 6, 0) to skip auto check_version requests on startup
2024-05-24 10:41:39,153 - group_id is None: disabling auto-commit.
2024-05-24 10:41:39,154 - Updating subscribed topics to: ('w',)
2024-05-24 10:41:39,159 - Updated partition assignment: [TopicPartition(topic='w', partition=0)]
2024-05-24 10:41:39,164 - <BrokerConnection node_id=1 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
2024-05-24 10:41:39,165 - <Brok

In [86]:
collected_data = [json.dumps(data) for data in collected_data]
collected_data = [json.loads(data) for data in collected_data]

# Normaize the data
datadf = pd.json_normalize(collected_data)

In [87]:
datadf

,gdp_per_capita,life_expectancy,freedom,corruption,generosity,year_2015,year_2016,year_2017,year_2018,year_2019
0,1.300000,0.999000,0.564000,0.151000,0.375000,0.0,0.0,0.0,0.0,1.0
1,0.670240,0.568440,0.177440,0.106130,0.111540,0.0,1.0,0.0,0.0,0.0
2,0.985490,0.602370,0.000000,0.137880,0.179220,1.0,0.0,0.0,0.0,0.0
3,0.422140,0.038240,0.128070,0.049520,0.186670,0.0,1.0,0.0,0.0,0.0
4,1.153184,0.709979,0.412730,0.132774,0.120990,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
152,0.488350,0.531190,0.434080,0.135090,0.259980,0.0,1.0,0.0,0.0,0.0
153,1.338000,0.698000,0.594000,0.123000,0.243000,0.0,0.0,0.0,1.0,0.0
154,1.343000,0.914000,0.677000,0.138000,0.353000,0.0,0.0,0.0,1.0,0.0
155,1.000820,0.685636,0.455198,0.140135,0.150112,0.0,0.0,1.0,0.0,0.0


In [88]:
predictions = model.predict(datadf)

datadf['predictions_score'] = predictions

In [89]:
datadf

,gdp_per_capita,life_expectancy,freedom,corruption,generosity,year_2015,year_2016,year_2017,year_2018,year_2019,predictions_score
0,1.300000,0.999000,0.564000,0.151000,0.375000,0.0,0.0,0.0,0.0,1.0,6.700403
1,0.670240,0.568440,0.177440,0.106130,0.111540,0.0,1.0,0.0,0.0,0.0,4.576819
2,0.985490,0.602370,0.000000,0.137880,0.179220,1.0,0.0,0.0,0.0,0.0,4.656019
3,0.422140,0.038240,0.128070,0.049520,0.186670,0.0,1.0,0.0,0.0,0.0,3.463084
4,1.153184,0.709979,0.412730,0.132774,0.120990,0.0,0.0,1.0,0.0,0.0,5.731495
...,...,...,...,...,...,...,...,...,...,...,...
152,0.488350,0.531190,0.434080,0.135090,0.259980,0.0,1.0,0.0,0.0,0.0,4.865342
153,1.338000,0.698000,0.594000,0.123000,0.243000,0.0,0.0,0.0,1.0,0.0,6.387005
154,1.343000,0.914000,0.677000,0.138000,0.353000,0.0,0.0,0.0,1.0,0.0,6.896936
155,1.000820,0.685636,0.455198,0.140135,0.150112,0.0,0.0,1.0,0.0,0.0,5.600661


In [90]:
csv_file_path = '../Data/ytest.csv'
df2 = pd.read_csv(csv_file_path)

In [91]:
datadf['score'] = df2['Score']

In [92]:
datadf

,gdp_per_capita,life_expectancy,freedom,corruption,generosity,year_2015,year_2016,year_2017,year_2018,year_2019,predictions_score,score
0,1.300000,0.999000,0.564000,0.151000,0.375000,0.0,0.0,0.0,0.0,1.0,6.700403,6.726
1,0.670240,0.568440,0.177440,0.106130,0.111540,0.0,1.0,0.0,0.0,0.0,4.576819,4.754
2,0.985490,0.602370,0.000000,0.137880,0.179220,1.0,0.0,0.0,0.0,0.0,4.656019,4.677
3,0.422140,0.038240,0.128070,0.049520,0.186670,0.0,1.0,0.0,0.0,0.0,3.463084,3.763
4,1.153184,0.709979,0.412730,0.132774,0.120990,0.0,0.0,1.0,0.0,0.0,5.731495,6.578
...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.488350,0.531190,0.434080,0.135090,0.259980,0.0,1.0,0.0,0.0,0.0,4.865342,4.996
153,1.338000,0.698000,0.594000,0.123000,0.243000,0.0,0.0,0.0,1.0,0.0,6.387005,6.105
154,1.343000,0.914000,0.677000,0.138000,0.353000,0.0,0.0,0.0,1.0,0.0,6.896936,7.495
155,1.000820,0.685636,0.455198,0.140135,0.150112,0.0,0.0,1.0,0.0,0.0,5.600661,6.008


In [93]:
with open('../Config/credentials.json', 'r') as json_file:
    credentials = json.load(json_file)

connection = mysql.connector.connect(
    host=credentials['host'],
    user=credentials['user'],
    password=credentials['password'],
)

connection

In [95]:
cursor = connection.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS WSFINAL")

cursor.execute("USE WSFINAL")
        
cursor.execute("""
            CREATE TABLE IF NOT EXISTS happiness_info (
                gdp_per_capita FLOAT,
                life_expectancy FLOAT,
                freedom FLOAT,
                corruption FLOAT,
                generosity FLOAT,
                year_2015 INT,
                year_2016 INT,
                year_2017 INT,
                year_2018 INT,
                year_2019 INT,
                prediction_score FLOAT,
                score FLOAT
            )
        """)

In [96]:
with open('../Config/credentials2.json', 'r') as json_file:
    keys = json.load(json_file)
    host = keys["host"]
    database = keys["database"]
    user = keys["user"]
    password = keys["password"]

database_connection = f"mysql+mysqlconnector://{user}:{password}@{host}/{database}"
engine = create_engine(database_connection)

datadf.to_sql(name='happiness_info', con=engine, if_exists='replace', index=False)

157